In [ ]:
version = "v2.2.033020"

# Assignment 2: Mining Itemsets (Part IV)

## Evaluating Frequent Itemsets

Even though we have found all the frequent itemsets, not all of them are interesting. In Part IV of this assignment, we will practice how to evaluate the frequent itemsets.

First, let's import the packages and dependencies that will be used in this part of assignment 2.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

**<span style="color:red">NOTE: These are all the imports we need to make for this assignment (Part IV). You should not make other imports in your submitted notebook. You will receive 0 points for the exercises if your solution includes additional imports.</span>**

People have developed various measurements of the interestingness of patterns. Most of them split the itemset into an antecedent item(set) and a consequent item(set), and then measure the correlation between the antecedent and the consequent. Let's try some of such measurements implemented by the `mlxtend.frequent_patterns.association_rules` API, which we have imported. For more information about the API, visit the [documentation](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/) of the `mlxtend` package.

Let's again use the shopping basket dataset as an example.

In [2]:
market_df = pd.read_csv('assets/shopping_basket.csv')
market_frequent_itemsets = apriori(market_df, min_support=0.005, use_colnames=True)

In [6]:
market_frequent_itemsets.sample(5)

,support,itemsets
304,0.005466,"(milk, energy_bar)"
599,0.005733,"(chocolate, ground_beef, frozen_vegetables)"
94,0.016531,(white_wine)
28,0.027063,(energy_bar)
537,0.011199,"(tomatoes, shrimp)"


In [5]:
interestingness_measurements = association_rules(market_frequent_itemsets, metric="lift", min_threshold=0)
interestingness_measurements.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(burgers),(almonds),0.087188,0.020397,0.005199,0.059633,2.923577,0.003421,1.041724
1,(almonds),(burgers),0.020397,0.087188,0.005199,0.254902,2.923577,0.003421,1.225089
2,(chocolate),(almonds),0.163845,0.020397,0.005999,0.036615,1.795099,0.002657,1.016834
3,(almonds),(chocolate),0.020397,0.163845,0.005999,0.294118,1.795099,0.002657,1.184553
4,(eggs),(almonds),0.179709,0.020397,0.006532,0.036350,1.782108,0.002867,1.016555


In the returned data frame, each row examines one (antecedent -> consequent) pair. *Antecedent support* and *consequent support* measure *P*(antecedent) and *P*(consequent), while *support* measures *P*(antecedent, consequent). In fact, these three values help us characterize the $2\times2$ contingency table, as illustrated in the following table:
 
 |           |              |       |              | 
 -----------:|:------------:|-------|---------------
 |           |    X = 1     | X = 0 |   sum(row)   |
 |     Y = 1 |   `support`    |       | `cons_support` |
 |     Y = 0 |              |       |              |
 | sum(col.) | `ante_support` |       |              |

Most interestingness measurements, including the four shown in the data frame (*confidence*, *lift*, *leverage*, and *conviction*), can be derived from the three support values. For example, $$\text{confidence}=\frac{\text{support}}{\text{antecedent_support}},$$ and $$\text{lift} =\frac{\text{confidence}}{\text{consequent_support}}=\frac{\text{support}}{\text{antecedent_support} * \text{consequent_support}}$$

### Exercise 5. (15 pts)
In this exercise, we are going to implement another interestingness measurement, the (full) mutual information, and add a 'mutual information' column to the data frame. The measurement is defined as

$$I(X;Y)=\sum_{x\in\mathcal{X}}\sum_{y\in\mathcal{Y}} P(X=x, Y=y)\log_2\frac{P(X=x,Y=y)}{P(X=x)P(P=y)}.$$

Note that the logorithm requirest that the joint probability $P(X=x, Y=y) > 0$, which does not hold for some $(x, y)$. However, since we know that when $P(X=x, Y=y) = 0$, it would not contribute to the sum, you may assume $P(X=x, Y=y)\log_2\frac{P(X=x,Y=y)}{P(X=x)P(P=y)} = 0$ in that case. 

$x$, $y$ are possible values of $X$ and $Y$; in the case of appearance or absence of an item, 1 or 0. Therefore, we need to consider all possible combinations of $x$ and $y$, that is, $(X=1, Y=1)$, $(X=1, Y=0)$, $(X=0, Y=1)$, $(X=0, Y=0)$.

Please complete the following function that uses the three support values to compute the mutual information. All the three parameters are in [0, 1], and you can assume the validity of the input. **Use 2 as the log base.** We have created some auxilary variables for you, each represent a joint or marginal (let $X$ denote the antecedent item and $Y$ denote the consequent item) probability.

In [9]:
def mi(antecedent_support, consequent_support, support):
       
    px1 = antecedent_support
    px0 = 1 - antecedent_support
    py1 = consequent_support
    py0 = 1 - consequent_support
    
    px1y1 = support
    px1y0 = px1 - px1y1
    px0y1 = py1 - px1y1
    px0y0 = 1 - px1 - py1 + px1y1
    
    mutual_information = 0
    if px1y1 != 0:
        mutual_information = mutual_information+ px1y1*np.log2(px1y1/px1/py1) 
    
    if px1y0 != 0:
        mutual_information = mutual_information+ px1y0*np.log2(px1y0/px1/py0) 
    
    if px0y1 != 0:
        mutual_information = mutual_information+ px0y1*np.log2(px0y1/px0/py1)
    
    if px0y0 != 0:
        mutual_information = mutual_information+ px0y0*np.log2(px0y0/px0/py0)
    
    
    return mutual_information      

In [10]:
# This code block tests whether the `mi` function work as expected.
# We hide some tests, so passing all the displayed assertions does not guarantee the bonus points.

assert np.abs(mi(0.6, 0.75, 0.4) - 0.04287484674660057) < 1e-8
assert np.abs(mi(0.5, 0.5, 0.25) - 0) < 1e-8

# If you fail the following assertion, double check if your function 
# handles the scenarios in which a joint probability is zero.
assert np.abs(mi(0.5, 0.5, 0.5) - 1) < 1e-8



Mutual information is a classical measure of interestingness. We encourage you to think about the following questions (not graded):
1. What is the maximum of mutual information in this setting? How to reach it?
2. What is the minimum of mutual information in this setting? How to reach it?

What does the max/min value imply?

With the `mi` function, we can now compute the mutual information for each (antecedent -> consequent) pair and attach it to the data frame. Does the result make sense?

In [11]:
interestingness_measurements['mi'] = \
    interestingness_measurements.apply(lambda pair: mi(pair['antecedent support'], 
                                              pair['consequent support'], 
                                              pair['support']),
                                       axis=1)
interestingness_measurements.sort_values('mi', ascending=False).head(n=5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,mi
676,(spaghetti),(ground_beef),0.174110,0.098254,0.039195,0.225115,2.291162,0.022088,1.163716,0.022631
677,(ground_beef),(spaghetti),0.098254,0.174110,0.039195,0.398915,2.291162,0.022088,1.373997,0.022631
655,(ground_beef),(herb_&_pepper),0.098254,0.049460,0.015998,0.162822,3.291994,0.011138,1.135410,0.014731
654,(herb_&_pepper),(ground_beef),0.049460,0.098254,0.015998,0.323450,3.291994,0.011138,1.332860,0.014731
1773,"(spaghetti, mineral_water)",(ground_beef),0.059725,0.098254,0.017064,0.285714,2.907928,0.011196,1.262445,0.013063
